In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df = pd.read_excel("/Users/leilingtao/Desktop/1860+美研金融专业16-21Fall申请实例.xlsx")

In [6]:
len(df)

1867

In [7]:
df.head()

,本科学校,专业,本科GPA,硕士院校,硕士专业,硕士GPA,IELTS,TOEFL,GMAT,GRE,入学年份,录取学校,学位,录取项目,录取结果,备注
0,浙大/复旦/南大/南开/武大/华中科大/上海交大/天大,经济学,3.53/4,NaN,NaN,NaN,NaN,Overall:107,NaN,Overall:328,2021,University of Wisconsin-Madison,MA,Financial Economics,Offer,课程：标准三件套+中微中宏+时序+面板，成绩都不怎么好 推荐信：本校海归老师*1，北美ap*...
1,双非,金融学,3.67 8/102,NaN,NaN,NaN,Overall:7 R: 7 / L: 7.5 / S: 6 / W: 6.5,NaN,NaN,NaN,2021,University of Illinois at Urbana-Champaign,MSc,finance,被拒,尽早申吧
2,中央财经大学,金融学,85.15/100,NaN,NaN,NaN,NaN,"Overall: 104, R: 28 / L: 28 / S: 20 / W: 28",NaN,"Overall: 328, V: 159 / Q: 169 / AW: 3",2021,Washington University in St. Louis,MS,Finance,offer,成绩前30%，三段实习经历，涵盖券商的各个部门，UCLA暑校经历；Early Turn拿一个...
3,211 & 985,传播+经济,3.9/4.3,NaN,NaN,NaN,Overall:7 R: 7.5 / L: 8 / S: 6 / W: 6.5,Overall:101 R: 27 / L: 29 / S: 23 / W: 22,NaN,NaN,2021,University of Rochester,MSc,Finance,Offer,1、尽早准备，尽早申请 2、申请与自己专业和实习相近的方向
4,对外经济贸易大学,小语种+金融,86.5,NaN,NaN,NaN,Overall:7.5 R: 8.5 / L: 8.5 / S: 6 / W: 6.5,Overall:100 R: 29 / L: 26 / S: 23 / W: 22,Overall:700 Q: 51 / V: 38,Overall:320 V: 152 / Q: 168 / AW: 3.5；LSAT:174,2021,University of California - San Diego,MSc,Finance,Waiting list,有关UZH的信息太少了，基本上说的最多的就是课程难不好毕业，相较而言总觉得UCSD课程设置更...


In [9]:
df['本科学校'].value_counts()

211          258
985          233
双非           159
211 & 985     36
西南财经大学        33
            ... 
加本             1
河北工业大学         1
华南理工学院         1
Top985         1
上海师范大学         1
Name: 本科学校, Length: 245, dtype: int64

In [2]:
import openai

In [9]:
import numpy as np

In [15]:
def run_gpt3(prompts):
    openai.api_key = ''

    engine="text-curie-001"
    temperature=0.1
    top_p=0.95

    response = openai.Completion.create(
        engine=engine,       
        prompt=prompts,
        temperature=temperature,
        top_p=top_p,
        echo=False,   # This function will not work
        logprobs=1,   # properly if any of these
        n=1,
        max_tokens=200)

    data = []
    for ex, prompt in zip(response["choices"], prompts):
        tokens = ex["logprobs"]["tokens"]
        logprobs = ex["logprobs"]["token_logprobs"]        
        probs = list(np.exp(logprobs))
        if "<|endoftext|>" in tokens:
            end_i = tokens.index("<|endoftext|>")
            tokens = tokens[ : end_i]  # This leaves off the "<|endoftext|>"
            probs = probs[ : end_i]    # token -- perhaps dubious.
        ans_indices = _find_generated_answer(tokens)
        answer_tokens = [tokens[i] for i in ans_indices]
        answer_probs = [probs[i] for i in ans_indices]
        answer = "".join(answer_tokens)        
        data.append({
            "prompt": prompt,
            "generated_text": ex["text"],
            "generated_tokens": tokens,
            "generated_probs": probs,
            "generated_answer": answer,
            "generated_answer_tokens": answer_tokens,
            "generated_answer_probs": answer_probs})
    return data

In [12]:
def _find_generated_answer(tokens, newline="\n" ): 
    """Our LMs tend to insert initial newline characters before
    they begin generating text. This function ensures that we 
    properly capture the true first line as the answer while
    also ensuring that token probabilities are aligned."""        
    answer_token_indices = []
    char_seen = False            
    for i, tok in enumerate(tokens):
        # This is the main condition: a newline that isn't an initial
        # string of newlines:
        if tok == newline and char_seen:
            break
        # Keep the initial newlines for consistency:
        elif tok == newline and not char_seen:
            answer_token_indices.append(i)
        # Proper tokens:
        elif tok != newline:
            char_seen = True
            answer_token_indices.append(i)
    return answer_token_indices 

In [5]:
prompts = ["我本科就读于中山大学经济学系，中山大学是985院校，我的绩点是3.7，我想申请美国的经济学类研究生，请问有什么选校建议？我的预算是每年3万美金。"]

In [18]:
prompts = ["I graduated from Sun yat-sen university，which is a top university in China. My GPA is 3.7/4.0 and my major is Economics. My tuition budget is $30000. Which American universities can I apply to for graduate study in economics?"]

In [21]:
prompts = ["Which second tier American universities offer economics graduate programs?"]

In [22]:
k = run_gpt3(prompts)

In [23]:
k

[{'prompt': 'Which second tier American universities offer economics graduate programs?',
  'generated_text': '\n\nThere are a number of second tier American universities that offer economics graduate programs. These universities may not be as well known as some of the top tier universities, but they offer excellent programs. Some of these universities include Boston University, George Washington University, and the University of Missouri.',
  'generated_tokens': ['\n',
   '\n',
   'There',
   ' are',
   ' a',
   ' number',
   ' of',
   ' second',
   ' tier',
   ' American',
   ' universities',
   ' that',
   ' offer',
   ' economics',
   ' graduate',
   ' programs',
   '.',
   ' These',
   ' universities',
   ' may',
   ' not',
   ' be',
   ' as',
   ' well',
   ' known',
   ' as',
   ' some',
   ' of',
   ' the',
   ' top',
   ' tier',
   ' universities',
   ',',
   ' but',
   ' they',
   ' offer',
   ' excellent',
   ' programs',
   '.',
   ' Some',
   ' of',
   ' these',
   ' unive